In [1]:

# Toy dataset
# Y  = [1, 1, 1,   1,   1,   0,   2,   0,   2,   0,   2,  1]
# Y  = [23, 25, 11, 14, 23, 18, 18, 14, 19, 5, 19, 16.5]
X1 = [1, 2, 5,   6,   3.5, 4,   2,   5.5, 2.5, 6.5, 1,  4]
X2 = [1, 2, 2.5, 3.5, 3,   5.5, 4.5, 5.5, 6,   7,   5,  1]

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X1, X2, Y)
ax.set_xlabel("X1")
ax.set_ylabel("X2")
ax.set_zlabel("Y")
plt.show()

# fig, ax = plt.subplots(1,1,figsize=(9,6))
# plot = ax.scatter(X1, X2, c=Y)
# ax.legend(*plot.legend_elements())
# plt.show()

data = pd.DataFrame({"X1": X1, "X2": X2, "Y": Y})
# X = data.drop("Y", axis=1).to_numpy()
# y = data["Y"].to_numpy()

# Build decision tree
DT = DecisionTree(data, y_type='continuous')

# Print structure
DT.print()

# Classify new example (X1=1, X2=6)
print(DT.predict([1, 6]))



NameError: name 'plt' is not defined